In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [2]:
df = pd.read_csv("stock_sentiment\stock_data.csv")
#df = df.rename(columns = {'Text':'text'})
test_text = df["Text"][10]
test_text

"i'm assuming FCX opens tomorrow above the 34.25 trigger buy. still very much like this setup.  "

In [3]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [5]:
json_data = prepare_json(test_text)
json_data

'{"signature_name": "serving_default", "instances": [{"examples": {"b64": "Cm0KawoEdGV4dBJjCmEKX2knbSBhc3N1bWluZyBGQ1ggb3BlbnMgdG9tb3Jyb3cgYWJvdmUgdGhlIDM0LjI1IHRyaWdnZXIgYnV5LiBzdGlsbCB2ZXJ5IG11Y2ggbGlrZSB0aGlzIHNldHVwLiAg"}}]}'

In [6]:
endpoint = "http://localhost:8080/v1/models/stock-sentiment-detection-model:predict"

In [7]:
response = requests.post(endpoint, data=json_data)
response

<Response [400]>